<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CONTEXTENGINEERING_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import google.generativeai as genai
import os
import time
import json
from typing import List, Dict, Any, Callable

# --- Agent Configuration and API Key Setup ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    EMBEDDING_MODEL_NAME: str = "text-embedding-004"
    MAX_TOKENS: int = 4096
    TEMPERATURE: float = 0.7

GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
    print('\n')
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the 'GEMINI' environment variable or Colab secret.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
embedding_model = genai.embed_content


print(f"LLM name: {AgentConfig.LLM_MODEL_NAME}")
print(f"EMBEDDING MODEL name: {AgentConfig.EMBEDDING_MODEL_NAME}")
print(f"Temperature: {AgentConfig.TEMPERATURE}")
print('\n')


# --- 1. Define Agent Tools (Simulated Functions) ---

# These functions simulate an agent calling out to external systems
def get_user_profile() -> Dict[str, Any]:
    """Retrieves the pilot's profile and preferences from a user database."""
    print("-> Calling tool: get_user_profile")
    return {
        "pilot_experience": "highly experienced",
        "priorities": ["fuel efficiency", "safety", "adherence to regulations"]
    }

def get_memory_data() -> List[str]:
    """Retrieves the pilot's recent flight history."""
    print("-> Calling tool: get_memory_data")
    return [
        "Recent trans-Pacific route with fuel-saving waypoints.",
        "Last flight used a high-altitude jet stream to reduce flight time."
    ]

def get_web_data() -> Dict[str, Any]:
    """Retrieves real-time data like weather and NOTAMs from web services."""
    print("-> Calling tool: get_web_data")
    return {
        "departure_airport": "JFK",
        "destination_airport": "LHR",
        "current_weather_jfk": "Clear skies, light winds.",
        "weather_en_route": "Developing storm system over the central Atlantic.",
        "notams": ["LHR_RUNWAY_09L_TEMPORARILY_CLOSED"]
    }

def get_rag_data() -> Dict[str, Any]:
    """Retrieves context from a knowledge base (e.g., aviation advisories)."""
    print("-> Calling tool: get_rag_data")
    return {
        "atc_advisories": "Aviation authority advisory suggests a more southern corridor for trans-Atlantic flights to avoid congestion and headwinds.",
        "alternative_routes": ["GANDER-2-OCEANIC_TRACK_B", "GANDER-3-OCEANIC_TRACK_D"]
    }

def get_mcp_data() -> Dict[str, Any]:
    """Retrieves aircraft-specific performance data."""
    print("-> Calling tool: get_mcp_data")
    return {
        "aircraft_type": "Boeing 747-8F",
        "optimal_altitude_for_payload": "37,000 feet",
        "max_fuel_consumption_rate": "15,000 kg/hour",
        "system_status": "All systems nominal, real-time engine monitoring enabled."
    }

def get_location_and_schedule_data() -> Dict[str, Any]:
    """Retrieves location-based context and schedule information."""
    print("-> Calling tool: get_location_and_schedule_data")
    return {
        "jfk_conditions": "Multiple departure runways available.",
        "lhr_conditions": "High volume of inbound cargo traffic expected during arrival window.",
        "departure_time": "August 4, 2025, 02:00 UTC",
        "arrival_deadline": "August 4, 2025, 12:00 UTC"
    }

# Map tool names to their functions
tools: Dict[str, Callable] = {
    "get_user_profile": get_user_profile,
    "get_memory_data": get_memory_data,
    "get_web_data": get_web_data,
    "get_rag_data": get_rag_data,
    "get_mcp_data": get_mcp_data,
    "get_location_and_schedule_data": get_location_and_schedule_data,
}

# --- 2. Main Agentic Execution Loop ---

def run_flight_planning_agent() -> str:
    """
    This function implements the agentic loop. It gives the LLM a goal
    and lets it "decide" which tools to call to gather information before
    generating the final output.
    """
    initial_prompt = (
        "You are an expert Flight Planning Agent. Your task is to generate the most efficient and safe flight plan for a long-haul cargo flight. "
        "Before you can create the plan, you must gather all necessary information. "
        "Here are the tools you have available:\n"
        f"{[tool for tool in tools.keys()]}\n\n"
        "To use a tool, respond with the name of the tool in the format: [CALL_TOOL:tool_name]. "
        "Once you have all the information, provide the final flight plan. "
        "Start by planning the flight from New York to London."
    )

    generation_config = genai.GenerationConfig(temperature=AgentConfig.TEMPERATURE)
    conversation_history = [{"role": "user", "parts": [initial_prompt]}]

    print("--- Starting Agentic Flight Planning Process ---")

    max_steps = 10  # Prevent infinite loops
    for step in range(max_steps):
        print(f"\n--- Agent Step {step + 1} ---")
        try:
            # Send the current conversation history to the LLM
            response = model.generate_content(
                conversation_history,
                generation_config=generation_config
            )
            llm_output = response.text.strip()
            print(f"LLM Output: {llm_output}")

            # Check if the LLM wants to call a tool
            if llm_output.startswith("[CALL_TOOL:"):
                tool_call = llm_output.replace("[CALL_TOOL:", "").replace("]", "").strip()
                if tool_call in tools:
                    tool_result = tools[tool_call]()
                    # Add the tool's result to the conversation history
                    conversation_history.append({"role": "model", "parts": [llm_output]})
                    conversation_history.append({"role": "user", "parts": [f"Tool Result: {json.dumps(tool_result)}"]})
                else:
                    # Handle a call to an undefined tool
                    conversation_history.append({"role": "model", "parts": [llm_output]})
                    conversation_history.append({"role": "user", "parts": ["Error: That tool does not exist. Please call a valid tool."]})
            else:
                # The LLM is providing the final response
                return llm_output
        except Exception as e:
            return f"An error occurred during the agentic process: {e}"

    return "Agent reached max steps without generating a final flight plan."


# --- 3. Main Execution Flow ---
if __name__ == "__main__":
    final_plan = run_flight_planning_agent()

    print("\n--- Final Generated Flight Plan ---")
    print(final_plan)

Google Generative AI configured successfully using Colab Secrets.


LLM name: gemini-2.5-flash
EMBEDDING MODEL name: text-embedding-004
Temperature: 0.7


--- Starting Agentic Flight Planning Process ---

--- Agent Step 1 ---
LLM Output: [CALL_TOOL:get_mcp_data]
-> Calling tool: get_mcp_data

--- Agent Step 2 ---
LLM Output: [CALL_TOOL:get_location_and_schedule_data]
-> Calling tool: get_location_and_schedule_data

--- Agent Step 3 ---
LLM Output: [CALL_TOOL:get_memory_data]
-> Calling tool: get_memory_data

--- Agent Step 4 ---
LLM Output: [CALL_TOOL:get_web_data]
-> Calling tool: get_web_data

--- Agent Step 5 ---
LLM Output: **Flight Plan: JFK (New York) to LHR (London)**

**1. Flight Identification:**
*   **Aircraft Type:** Boeing 747-8F (Cargo)
*   **Origin:** John F. Kennedy International Airport (JFK), New York, USA
*   **Destination:** London Heathrow Airport (LHR), London, UK

**2. Departure Plan (JFK):**
*   **Scheduled Departure Time:** August 4, 2025, 02:00 UTC
*   **Condit